In [9]:
def get_bigquery_client():
    """Função para autenticar e criar o cliente do BigQuery"""
    from google.cloud import bigquery

    client = bigquery.Client()
    return client

In [17]:
def get_drive_service():
    """Função para autenticar e criar o serviço do Google Drive"""
    from googleapiclient.discovery import build

    service = build("drive", "v3")
    return service

In [11]:
def get_sheets_service():
    """Função para autenticar e criar o serviço do Google Sheets"""
    from googleapiclient.discovery import build

    service = build('sheets', 'v4')
    return service

In [12]:
def create_new_spreadsheet(service, title="Nova Planilha"):
    """Função para criar uma nova planilha"""
    from googleapiclient.errors import HttpError

    spreadsheet_body = {
        "properties": {"title": title}
    }
    try:
        spreadsheet = service.spreadsheets().create(body=spreadsheet_body).execute()
        print(f"Planilha criada com sucesso: {spreadsheet['spreadsheetId']}")
        return spreadsheet['spreadsheetId']
    except HttpError as err:
        print(f"Erro ao criar a planilha: {err}")
        raise

In [13]:
def share_file_with_email(drive_service, file_id, email, role="writer"):
    """Função para compartilhar o arquivo com um e-mail específico"""
    from googleapiclient.errors import HttpError

    permissions = {
        "type": "user",
        "role": role,  # "writer" para editor ou "reader" para visualizador
        "emailAddress": email
    }
    try:
        drive_service.permissions().create(
            fileId=file_id,
            body=permissions,
            fields="id"
        ).execute()
        print(f"Arquivo compartilhado com sucesso com {email} como {role}.")
    except HttpError as err:
        print(f"Erro ao compartilhar o arquivo: {err}")
        raise

In [14]:
def upload_to_sheets(service, spreadsheet_id, range_name, df):
    """Função para enviar os dados do DataFrame para o Google Sheets"""
    from googleapiclient.errors import HttpError

    # Converte o DataFrame para uma lista de listas (cada linha é uma lista)
    values = [df.columns.values.tolist()] + df.values.tolist()
    
    body = {
        'values': values
    }
    
    try:
        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption="RAW",
            body=body
        ).execute()
        print(f"{result.get('updatedCells')} células atualizadas.")
    except HttpError as err:
        print(f"Erro ao atualizar a planilha: {err}")
        raise

In [15]:
def run_bigquery_query(client, query):
    """Função para executar uma consulta no BigQuery e retornar os resultados como um DataFrame"""
    query_job = client.query(query)
    results = query_job.result()
    # Converte os resultados para um DataFrame do pandas
    df = results.to_dataframe()
    return df

In [18]:
import os

# Configuração

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "configs/credentials.json"
spreadsheet_id = "1V8AsvW6Z6zQCE5Gedhbn-yQFxjeYRe3ynlyXXpZ3XPw" #None  # Defina como None para criar um novo documento se necessário
range_name = "Sheet1!A1"  # O intervalo onde os dados serão escritos (exemplo: 'Sheet1!A1')
email_to_share = "jeffersonkakashi@gmail.com"

# Query para buscar os dados do BigQuery
query = """
SELECT *
FROM `beAnalytics.steam_sales`
"""

# 1. Conectando-se ao BigQuery
client = get_bigquery_client()

# 2. Executando a consulta no BigQuery
df = run_bigquery_query(client, query)

# 3. Conectando-se ao Google Sheets
drive_service = get_drive_service()

# 4. Conectando-se ao Google Sheets
sheets_service = get_sheets_service()

# 5. Verificando ou criando a planilha
if not spreadsheet_id:
    spreadsheet_id = create_new_spreadsheet(sheets_service, title="Steam")

# 6. Compartilhar a planilha
share_file_with_email(drive_service, spreadsheet_id, email_to_share, role="writer")

# 7. Enviando os dados para o Google Sheets
upload_to_sheets(sheets_service, spreadsheet_id, range_name, df)

/workspaces/beAnalytics_case/beanalytics_venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Arquivo compartilhado com sucesso com jeffersonkakashi@gmail.com como writer.
9398 células atualizadas.
